In [ ]:
!pip install segment-anything opencv-python pytesseract transformers easyocr --quiet
!pip install git+https://github.com/facebookresearch/segment-anything.git --quiet
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth -O sam_vit_b.pth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
--2026-02-23 01:53:50--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.249.182.81, 13.249.182.62, 13.249.182.39, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.249.182.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375042383 (358M) [binary/octet-stream]
Saving to: ‘sam_vit_b.pth’

sam_vit_b.pth       100%[===================>] 357.67M   245MB/s    in 1.5s    

2026-02-23 01:53:52 (245 MB/s) - ‘sam_vit_b.pth’ saved [375042383/375042383]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import zipfile
import os


zip_path = "/content/drive/MyDrive/Colab Notebooks/MNA/Proyecto Integrador/Copia de 20260121 imagenes Cashcollection1.zip"
extract_dir = "/content/drive/MyDrive/Colab Notebooks/MNA/Proyecto Integrador/dataset_bimbonet"

# Extraer ZIP
with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(extract_dir)

print("ZIP extraído en:", extract_dir)

# Carpeta donde están las imágenes
IMAGE_DIR = os.path.join(extract_dir, "20260121 imagenes Cashcollection")
print("Carpeta de imágenes:", IMAGE_DIR)

# Verificar 5 archivos
imgs = os.listdir(IMAGE_DIR)[:5]
print("Ejemplo de archivos:", imgs)

Mounted at /content/drive
ZIP extraído en: /content/drive/MyDrive/Colab Notebooks/MNA/Proyecto Integrador/dataset_bimbonet
Carpeta de imágenes: /content/drive/MyDrive/Colab Notebooks/MNA/Proyecto Integrador/dataset_bimbonet/20260121 imagenes Cashcollection
Ejemplo de archivos: ['base64decoded (61).png', 'base64decoded (60).png', 'base64decoded (63).png', 'base64decoded (62).png', 'base64decoded (64).png']


In [ ]:
import cv2
import torch
import numpy as np
import pytesseract
import re
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator

from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection
import easyocr

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", device)

# SAM
sam = sam_model_registry["vit_b"](checkpoint="sam_vit_b.pth").to(device)

mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=4,
    pred_iou_thresh=0.88,
    stability_score_thresh=0.90,
    min_mask_region_area=300
)

# DINO
processor = AutoProcessor.from_pretrained("IDEA-Research/grounding-dino-base")
dino = AutoModelForZeroShotObjectDetection.from_pretrained(
    "IDEA-Research/grounding-dino-base"
).to(device)

# EasyOCR como OCR alternativo
reader = easyocr.Reader(['es', 'en'], gpu=(device=="cuda"))

DEVICE: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/457 [00:00<?, ?B/s]

The image processor of type `GroundingDinoImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


config.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/933M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/1206 [00:00<?, ?it/s]

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [ ]:
def extract_col_strict(text):
    """
    Extrae patrones COL con robustez estilo DINO.
    """
    text = text.upper()
    pattern = r"C[O0]L\s*([123])[\.\,\s]*(\d{5})\s+(\d{5})"
    matches = re.findall(pattern, text)
    return [f"COL {m[0]}. {m[1]} {m[2]}" for m in matches]

In [ ]:
def sam_extract(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_small = cv2.resize(img_rgb, (1024, 1024))

    masks = mask_generator.generate(img_small)
    found = []

    for m in masks:
        mask = m["segmentation"].astype(np.uint8) * 255
        x, y, w, h = cv2.boundingRect(mask)
        crop = img_small[y:y+h, x:x+w]

        if crop.size == 0:
            continue

        ocr = pytesseract.image_to_string(crop)
        cleaned = extract_col_strict(ocr)
        if cleaned:
            found.extend(cleaned)

    return sorted(list(set(found)))

In [ ]:
def dino_fallback(img):
    if img is None or img.size == 0:
        return []

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # ---- DINO inference ----
    inputs = processor(images=img_rgb, text="printed text with numbers", return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = dino(**inputs)

    results = processor.post_process_grounded_object_detection(
        outputs, inputs.input_ids,
        threshold=0.22, text_threshold=0.22,
        target_sizes=[img.shape[:2]]
    )[0]

    found = []
    h, w = img.shape[:2]

    # ---- Procesar cada caja detectada ----
    for box in results["boxes"]:
        x1, y1, x2, y2 = map(int, box.tolist())

        # Protección contra coordenadas inválidas
        x1 = max(0, min(x1, w - 1))
        x2 = max(0, min(x2, w - 1))
        y1 = max(0, min(y1, h - 1))
        y2 = max(0, min(y2, h - 1))

        # Evitar crops vacíos
        if x2 <= x1 or y2 <= y1:
            continue

        crop = img[y1:y2, x1:x2]

        # Evitar errores de tamaño 0
        if crop is None or crop.size == 0:
            continue

        try:
            # OCR seguro con EasyOCR
            ocr_list = reader.readtext(crop, detail=0)
        except:
            continue

        text = " ".join(ocr_list)
        cleaned = extract_col_strict(text)

        if cleaned:
            found.extend(cleaned)

    return sorted(list(set(found)))

In [ ]:
def extract_col_hybrid(img_path):
    img = cv2.imread(img_path)

    # 1) SAM rápido
    sam_result = sam_extract(img)
    if sam_result:
        return sam_result

    # 2) Si SAM falla → usar DINO
    dino_result = dino_fallback(img)
    return dino_result if dino_result else []

In [ ]:
results = {}

for img_name in os.listdir(IMAGE_DIR):
    if not img_name.lower().endswith((".jpg", ".jpeg", ".png", ".jfif")):
        continue

    path = os.path.join(IMAGE_DIR, img_name)
    detected = extract_col_hybrid(path)
    results[img_name] = detected
    print(img_name, "→", detected)

base64decoded (61).png → ['COL 1. 90518 99153', 'COL 2. 60119 61540']
base64decoded (60).png → ['COL 1. 07827 91841']
base64decoded (63).png → ['COL 1. 63526 60059', 'COL 2. 33107 90542', 'COL 3. 91319 39228']
base64decoded (62).png → ['COL 1. 99501 93351', 'COL 2. 30437 63553', 'COL 3. 24563 56080']
base64decoded (64).png → ['COL 1. 28535 92494', 'COL 2. 81347 04590', 'COL 3. 93836 13295']
base64decoded (65).png → ['COL 1. 81494 20104', 'COL 2. 20115 91441', 'COL 3. 81511 88589']
base64decoded (66).png → ['COL 1. 96353 46105']
base64decoded (67).png → ['COL 1. 53558 89054', 'COL 2. 60009 60573', 'COL 3. 60359 54392']
base64decoded (68).png → ['COL 2. 51608 50516']
base64decoded (69).png → ['COL 1. 70568 09735', 'COL 2. 81010 61571', 'COL 2. 81610 61571', 'COL 3. 90557 14700', 'COL 3. 90557 14709']
base64decoded (7).png → ['COL 1. 49457 35838', 'COL 2. 40063 66475', 'COL 2. 40083 66475', 'COL 3. 60850 89317']
base64decoded (70).png → ['COL 2. 00019 61472']
base64decoded (71).png → ['CO